# Tokocrypto to CoinTracker
## Installation

In [1]:
import pandas as pd
import datetime

In [2]:
df_tko = pd.read_excel('./data/tokocrypto.xls')

## Example Data Preview

In [3]:
df_tko.head()

,Time,Pair,Side,Price,Filled,Trading Fees,Total
0,10-30 16:59:18,COTI/USDT,Buy,0.57,31.0,0.031 COTI,17.67 USDT
1,10-30 12:55:51,USDT/BIDR,Buy,"14,176",18.0,0.018 USDT,"255,168 BIDR"
2,10-29 21:22:07,TKO/BIDR,Buy,"26,960",9.2,0.0092 TKO,"248,032 BIDR"
3,10-29 11:04:24,FTM/BIDR,Buy,"42,210",11.8,0.0118 FTM,"498,078 BIDR"
4,10-28 16:08:14,FTM/BIDR,Buy,"47,001",4.3,0.0043 FTM,"202,104.3 BIDR"


In [4]:
df_tko.dtypes

Time             object
Pair             object
Side             object
Price            object
Filled          float64
Trading Fees     object
Total            object
dtype: object

## Example data final format

In [5]:
df_ct = pd.read_csv('./data/cointracker.csv')
df_ct.head()

,Date,Received Quantity,Received Currency,Sent Quantity,Sent Currency,Fee Amount,Fee Currency,Tag
0,06/14/2017 20:57:35,0.5,BTC,4005.8,USD,0.00001,BTC,NaN
1,08/19/2017 10:05:15,0.3,BTC,3.0,ETH,NaN,NaN,NaN
2,08/21/2017 12:00:00,NaN,NaN,3.0,ETH,0.00010,ETH,gift
3,08/30/2017 12:01:30,3.0,ETH,NaN,NaN,NaN,NaN,mined


In [6]:
df_ct.dtypes  

Date                  object
Received Quantity    float64
Received Currency     object
Sent Quantity        float64
Sent Currency         object
Fee Amount           float64
Fee Currency          object
Tag                   object
dtype: object

In [7]:
df_ct['Date'] = pd.to_datetime(df_ct['Date'])

## Converting Tokocrypto CSV to CoinTracker
1. Data Cleaning
  - Convert time to datatime
  - Separate `Pair` (ex: TKO/BIDR)
    - -> `Pair From` (ex: BIDR)
    - -> `Pair To` (ex: TKO)
  - Separate token with its value
    - `Trading Fees` (ex: 0.0092 TKO)
      - -> `Fee Currency` (ex: TKO)
      - -> `Fee Amount` (ex: 0.0092)
    - `Total` (ex: 0.0092 TKO)
      - -> `Total Currency` (ex: TKO)
      - -> `Total Amount` (ex: 0.0092)
  - String to Int
    - `Price`
    - `Total Amount`
    - `Fee Amount`
2. Assign to correct column
  - `Date`: Time
  - `Received Quantity`: ((Price * Filled) - Trading Fees)
  - `Received Currency`: Pair To
  - `Sent Quantity`: Total Amount
  - `Sent Currency`: Total Currency
  - `Fee Amount`: Fee Amount
  - `Fee Currency`: Fee Currency
  - `Tag`: NaN

## Data Cleaning

### Convert time to datatime

In [8]:
def fix_year(date):
  invalid_year = 1900
  this_year = datetime.datetime.now().year
  if date.year == invalid_year:
    date = date.replace(year = this_year)
  return date


In [9]:
df_tko['Time'] = pd.to_datetime(df_tko['Time'], format='%m-%d %H:%M:%S')
df_tko['Time'] = df_tko['Time'].apply(fix_year)

### Separate Pair
Separate `Pair` (ex: TKO/BIDR)
- -> `Pair From` (ex: BIDR)
- -> `Pair To` (ex: TKO)

In [10]:
df_tko[['Pair To', 'Pair From']] = df_tko['Pair'].str.split('/', 1, expand = True)

### Separate token with its value
- `Trading Fees` (ex: 0.0092 TKO)
  - -> `Fee Currency` (ex: TKO)
  - -> `Fee Amount` (ex: 0.0092)
- `Total` (ex: 0.0092 TKO)
  - -> `Total Currency` (ex: TKO)
  - -> `Total Amount` (ex: 0.0092)

In [11]:
df_tko[['Fee Amount', 'Fee Currency']] = df_tko['Trading Fees'].str.split(' ', 1, expand = True)
df_tko[['Total Amount', 'Total Currency']] = df_tko['Total'].str.split(' ', 1, expand = True)

### String to float
- `Price`
- `Total Amount`
- `Fee Amount`

In [12]:
def convert_string_to_float(series):
  return series\
    .apply(lambda x: x.replace(',', ''))\
    .astype(float)

In [13]:
df_tko['Price'] = convert_string_to_float(df_tko['Price'])
df_tko['Total Amount'] = convert_string_to_float(df_tko['Total Amount'])
df_tko['Fee Amount'] = convert_string_to_float(df_tko['Fee Amount'])

## Assign to correct Columns
- `Date`: Time
- `Received Quantity`: ((Price * Filled) - Fee Amount)
- `Received Currency`: Pair To
- `Sent Quantity`: Total Amount
- `Sent Currency`: Total Currency
- `Fee Amount`: Fee Amount
- `Fee Currency`: Fee Currency
- `Tag`: NaN

### DataFrame

In [15]:
df_result = pd.DataFrame({
  'Date': df_tko['Time'],
  'Received Quantity': df_tko['Filled'],
  'Received Currency': df_tko['Pair To'],
  'Sent Quantity': df_tko['Total Amount'],
  'Sent Currency': df_tko['Total Currency'],
  'Fee Amount': df_tko['Fee Amount'],
  'Fee Currency': df_tko['Fee Currency'],
  'Tag': ''
})

In [16]:
df_result.set_index('Date', inplace=True) 

In [ ]:
df_result.head()

,Received Quantity,Received Currency,Sent Quantity,Sent Currency,Fee Amount,Fee Currency,Tag
Date,,,,,,,
2021-10-30 16:59:18,30.9690,COTI,17.67,USDT,0.0310,COTI,
2021-10-30 12:55:51,17.9820,USDT,255168.00,BIDR,0.0180,USDT,
2021-10-29 21:22:07,9.1908,TKO,248032.00,BIDR,0.0092,TKO,
2021-10-29 11:04:24,11.7882,FTM,498078.00,BIDR,0.0118,FTM,
2021-10-28 16:08:14,4.2957,FTM,202104.30,BIDR,0.0043,FTM,


## Export Data

In [ ]:
filename = 'tokocrypto_cointracker_{datetime}.csv'.format(datetime = datetime.datetime.now())
directory = './data/{filename}'.format(filename = filename)
df_result.to_csv(directory, sep=',', encoding='utf-8')